This notebook is exploratory data analysis of the different datasets.  We look to see:

- what data is available in each dataset
- if there are any missing values and/or outliers
- summary statistics on some of the data

In [1]:
import pandas as pd

# Set up GCP and AWS bucket APIs
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

# US Census ACS data

First we see what's available in ACS and load one file.

# Set up BigQuery column info queries and DB/Table names

In [46]:
# ACS US census data
ACS_DB = '`bigquery-public-data`.census_bureau_acs'
ACS_TABLE = 'zip_codes_2017_5yr'

# project sunroof
PSR_DB = '`bigquery-public-data`.sunroof_solar'
PSR_TABLE = 'solar_potential_by_postal_code'

# This gives us columns:
# ['table_catalog', 'table_schema', 'table_name', 'column_name',
# 'field_path', 'data_type', 'description']
column_descr_query = """SELECT *
FROM {}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE table_name="{}";
"""

# we don't get description, but get IS_NULLABLE from this
column_details_query = """SELECT *
FROM {}.INFORMATION_SCHEMA.COLUMNS
WHERE table_name="{}";
"""

# Examine ACS schema

GBQ reference for getting metadata on tables: 

In [47]:
query = column_descr_query.format(ACS_DB, ACS_TABLE)

df = pd.read_gbq(query)

Downloading: 100%|██████████| 252/252 [00:00<00:00, 1373.85rows/s]


In [48]:
# show all rows
pd.set_option('display.max_rows', None)

In [49]:
df[['column_name', 'data_type', 'description']]

,column_name,data_type,description
0,geo_id,STRING,US Census Zip Code Tabulation Areas Geoids
1,nonfamily_households,FLOAT64,Nonfamily Households. A householder living alo...
2,family_households,FLOAT64,Family Households. A family consists of a hous...
3,median_year_structure_built,FLOAT64,Median Year Structure Built. Median Year Struc...
4,rent_burden_not_computed,FLOAT64,Housing units without rent burden computed. Un...
5,rent_over_50_percent,FLOAT64,Housing units spending over 50% income on rent...
6,rent_40_to_50_percent,FLOAT64,Housing units spending 40 to 49.9% income on r...
7,rent_35_to_40_percent,FLOAT64,Housing units spending 35 to 39.9% income on r...
8,rent_30_to_35_percent,FLOAT64,Housing units spending 30 to 34.9% income on r...
9,rent_25_to_30_percent,FLOAT64,Housing units spending 25 to 29.9% income on r...


How many rows in the ACS census data table?

In [50]:
query = f"""SELECT COUNT(*)
FROM {ACS_DB}.{ACS_TABLE}"""

query_job = client.query(query)

In [51]:
list(query_job.result())[0][0]

33120

How many unique zip codes?

In [52]:
query = f"""SELECT COUNT(DISTINCT geo_id)
FROM {ACS_DB}.{ACS_TABLE}"""

query_job = client.query(query)

In [53]:
list(query_job.result())[0][0]

33120

In [54]:
# columns to keep from ACS data
ACS_COLS = ['geo_id',
            'total_pop',
            'median_age'
            'housing_units',
            'median_income',
            'housing_units',
            'owner_occupied_housing_units',
            'occupied_housing_units',
            # housing units which will be used to calculate total single-family homes
           'dwellings_1_units_detached',
           'dwellings_1_units_attached',
           'dwellings_2_units',
           'dwellings_3_to_4_units',
           'bachelors_degree_2',
           'different_house_year_ago_different_city',
           'different_house_year_ago_same_city',
           'gini_index']

# Explore Project Sunroof Data from BigQuery

In [55]:
query = column_descr_query.format(PSR_DB, PSR_TABLE)
df = pd.read_gbq(query)
df

Downloading: 100%|██████████| 32/32 [00:00<00:00, 242.62rows/s]


,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,region_name,region_name,STRING,Postal Code
1,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,state_name,state_name,STRING,Name of the state containing that region
2,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,lat_max,lat_max,FLOAT64,maximum latitude for that region
3,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,lat_min,lat_min,FLOAT64,minimum latitude for that region
4,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,lng_max,lng_max,FLOAT64,maximum longitude for that region
5,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,lng_min,lng_min,FLOAT64,minimum longitude for that region
6,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,lat_avg,lat_avg,FLOAT64,average latitude for that region
7,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,lng_avg,lng_avg,FLOAT64,average longitude for that region
8,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,yearly_sunlight_kwh_kw_threshold_avg,yearly_sunlight_kwh_kw_threshold_avg,FLOAT64,75% of the optimimum sunlight in the county co...
9,bigquery-public-data,sunroof_solar,solar_potential_by_postal_code,count_qualified,count_qualified,INT64,# of buildings in Google Maps that are suitabl...


How many rows and unique zip codes here?

In [56]:
query = f"SELECT COUNT(*) FROM {PSR_DB}.{PSR_TABLE};"
df = pd.read_gbq(query)
df

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.54rows/s]


,f0_
0,11516


In [58]:
query = f"SELECT COUNT(DISTINCT region_name) FROM {PSR_DB}.{PSR_TABLE};"
df = pd.read_gbq(query)
df

Downloading: 100%|██████████| 1/1 [00:00<00:00,  6.47rows/s]


,f0_
0,11512


Hmm, some duplicates or something?

# LBNL solar install data



In [73]:
df1 = pd.read_csv('data/TTS_LBNL_public_file_10-Dec-2019_p1.csv', encoding='latin-1')

/home/nate/anaconda3/envs/dend_capstone/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,2,19,20,32,34,35,36,37,39,40,41,48,49,50,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
df2 = pd.read_csv('data/TTS_LBNL_public_file_10-Dec-2019_p2.csv', encoding='latin-1')

/home/nate/anaconda3/envs/dend_capstone/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,2,19,20,22,24,32,33,34,35,36,37,39,40,41,48,49,50,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
lbnl_df = pd.concat([df1, df2], axis=0)

In [80]:
lbnl_df.shape

(1543831, 60)

In [103]:
# this query takes a while to run, but it ends up being 2.7GB of memory use
lbnl_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1543831 entries, 0 to 843830
Data columns (total 60 columns):
 #   Column                                                Non-Null Count    Dtype  
---  ------                                                --------------    -----  
 0   Data Provider                                         1543831 non-null  object 
 1   System ID (from first Data Provider)                  1543831 non-null  object 
 2   System ID (from second Data Provider, if applicable)  1543831 non-null  object 
 3   System ID (Tracking the Sun)                          1543831 non-null  object 
 4   Installation Date                                     1543831 non-null  object 
 5   System Size                                           1543831 non-null  float64
 6   Total Installed Price                                 1543831 non-null  float64
 7   Appraised Value Flag                                  1543831 non-null  bool   
 8   Sales Tax Cost                   

In [90]:
lbnl_df.columns

Index(['Data Provider', 'System ID (from first Data Provider)',
       'System ID (from second Data Provider, if applicable)',
       'System ID (Tracking the Sun)', 'Installation Date', 'System Size',
       'Total Installed Price', 'Appraised Value Flag', 'Sales Tax Cost',
       'Rebate or Grant', 'Performance-Based Incentive (Annual Payment)',
       'Performance-Based Incentives (Duration)',
       'Feed-in Tariff (Annual Payment)', 'Feed-in Tariff (Duration)',
       'Customer Segment', 'New Construction', 'Tracking', 'Ground Mounted',
       'Battery System', 'Zip Code', 'City', 'State',
       'Utility Service Territory', 'Third-Party Owned', 'Installer Name',
       'Self-Installed', 'Azimuth #1', 'Azimuth #2', 'Azimuth #3', 'Tilt #1',
       'Tilt #2', 'Tilt #3', 'Module Manufacturer #1', 'Module Model #1',
       'Module Manufacturer #2', 'Module Model #2', 'Module Manufacturer #3',
       'Module Model #3', 'Additional module model', 'Module Technology #1',
       'Module T

In [91]:
lbnl_df[['Module Manufacturer #1', 'Module Manufacturer #2', 'Module Manufacturer #3']].head()

,Module Manufacturer #1,Module Manufacturer #2,Module Manufacturer #3
0,Sharp,-9999,-9999
1,Sharp,-9999,-9999
2,Sharp,-9999,-9999
3,Sharp,-9999,-9999
4,Sharp,-9999,-9999


How many residential installs in the data?

In [81]:
lbnl_df[lbnl_df['Customer Segment'] == 'RES'].shape

(1456225, 60)

Who are the top installers?

Note: -9999 is missing data.

In [83]:
lbnl_df['Installer Name'].value_counts()

solarcity                                                                                      283374
-9999                                                                                           92562
vivint                                                                                          90032
sunrun                                                                                          54550
-9999                                                                                           49152
sunpower                                                                                        40587
trinity                                                                                         32865
sungevity                                                                                       24227
petersen dean                                                                                   22466
verengo                                                                           

How many self-installed?

In [92]:
lbnl_df['Self-Installed'].value_counts()

 0       1261516
-9999     260025
 1         22290
Name: Self-Installed, dtype: int64

How many datapoints missing self-installed and installer?

In [95]:
lbnl_df[(lbnl_df['Self-Installed'] == -9999) & (lbnl_df['Installer Name'] == -9999)].shape

(49152, 60)

# EIA-861 Report

Energy prices and amount sold to residential customers.

In [87]:
eia861_df = pd.read_excel('data/Sales_Ult_Cust_2018.xlsx')

In [88]:
eia861_df.shape

(3436, 24)

In [106]:
eia861_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3436 entries, 0 to 3435
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Utility Characteristics  3435 non-null   object
 1   Unnamed: 1               3434 non-null   object
 2   Unnamed: 2               3434 non-null   object
 3   Unnamed: 3               3434 non-null   object
 4   Unnamed: 4               3434 non-null   object
 5   Unnamed: 5               3434 non-null   object
 6   Unnamed: 6               3434 non-null   object
 7   Unnamed: 7               3275 non-null   object
 8   Unnamed: 8               3403 non-null   object
 9   RESIDENTIAL              3435 non-null   object
 10  Unnamed: 10              3435 non-null   object
 11  Unnamed: 11              3435 non-null   object
 12  COMMERCIAL               3435 non-null   object
 13  Unnamed: 13              3435 non-null   object
 14  Unnamed: 14              3435 non-null  

# EIA IOU rates with zipcodes

In [89]:
iou_df = pd.read_csv('data/iouzipcodes2017.csv')
noniou_df = pd.read_csv('data/noniouzipcodes2017.csv')
eia_zipcode_df = pd.concat([iou_df, noniou_df], axis=0)
eia_zipcode_df.shape

(86672, 9)

In [105]:
eia_zipcode_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86672 entries, 0 to 34073
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           86672 non-null  int64  
 1   eiaid         86672 non-null  int64  
 2   utility_name  86672 non-null  object 
 3   state         86672 non-null  object 
 4   service_type  86672 non-null  object 
 5   ownership     86672 non-null  object 
 6   comm_rate     86672 non-null  float64
 7   ind_rate      86672 non-null  float64
 8   res_rate      86672 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 26.7 MB
